In [1]:
import sys
sys.path.append("../src/")
sys.path.append("..")

from nltk.corpus import semcor
#from nltk.tree import Tree
#import itertools
#import random
import pandas as pd
import torch
from bert import *
from feature_data import *
from multiprototype import *
import classifier_main
import csv
from nltk.corpus.reader.wordnet import Lemma
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

import inflect
import os
from scipy.stats import spearmanr, pearsonr
import re
from src.utils import *



In [2]:
bert = BERTBase()


INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmp1yar9g4v
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [46]:
# first sample 5 random words from the test set

feature_norms = McRaeFeatureNorms('../data/external/mcrae/CONCS_FEATS_concstats_brm/concepts_features-Table1.csv')
layer = 8
clusters = 5
embedding_file = '../data/processed/multipro_embeddings/layer'+ str(layer) + 'clusters' + str(clusters) + '.txt'
embs = read_multiprototype_embeddings(embedding_file, layer=layer, num_clusters=clusters)

train, dev, test = classifier_main.prepare_data(feature_norms, embs)

test_words = []
for word in test:
    test_words.append(word)

test_words

Read in 12348 vectors of size 5 X 768
Starting with 433 training words


['coconut',
 'crayon',
 'calf',
 'yacht',
 'potato',
 'pier',
 'blackbird',
 'spear',
 'microscope',
 'cow',
 'projector',
 'drill',
 'nylons',
 'bus',
 'shelves',
 'cake',
 'crowbar',
 'bullet',
 'lamb',
 'stereo',
 'stool_(furniture)',
 'rocket',
 'belt',
 'violin',
 'walnut',
 'clock',
 'alligator',
 'tongs',
 'taxi',
 'bucket',
 'ambulance',
 'swimsuit',
 'pants',
 'camisole',
 'mat',
 'bread',
 'door',
 'bull',
 'knife',
 'coyote',
 'cod',
 'grapefruit',
 'lobster',
 'strainer',
 'helicopter',
 'cauliflower',
 'tank_(army)',
 'house',
 'chisel',
 'blueberry',
 'bison',
 'robe',
 'cupboard',
 'card_(greeting)']

In [47]:
from random import sample
sample(test_words, 5)

['helicopter', 'bus', 'stool_(furniture)', 'lobster', 'strainer']

In [48]:
words = ['lamb', 'bullet', 'crayon', 'nylons', 'blackbird']

In [50]:
# select models to look at- only mcrae and binder datasets, only bert 1k 

# MCRAE

models = [
    '../trained_models/model.ffnn.mc_rae_real.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.ffnn.mc_rae_real.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.mc_rae_real.1k.50components.500max_iters',
    #'../trained_models/model.plsr.mc_rae_real.glove.100components.500max_iters',
    '../trained_models/model.modabs.mc_rae_real.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4',
    #'../trained_models/model.modabs.mc_rae_real.glove.mu1_1.mu2_0.1.mu3_1e-07.mu4_5.nnk_5',
    #'../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.plsr.binder.1k.300components.500max_iters',
    #'../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]


In [51]:
records = []

for word in words:
    print("****************")
    print("gold features for %s" % word)
    print("****************")
    golds = feature_norms.get_features(word)
    print(golds)
    record = {}
    record["word"] = word
    record["model"] = "gold"
    for i in range(0, len(golds)):
        gold = golds[i]
        record["feat_" + str(i)] = gold
    records.append(record)    
    
    print("****************")
    print("predictions for %s" % word)
    print("****************")
    for save_path in models:
        model = torch.load(save_path)
        print(save_path)

        record = {}
        record["word"] = word
        record["model"] = save_path.split(".")[3]
        feats = model.predict_top_n_features(word, 10)
        for i in range(0, len(feats)):
            feat = feats[i]
            record["feat_" + str(i)] = feat
            
            print("     ", feat)
        records.append(record)

****************
gold features for lamb
****************
['is_white', 'beh_-_bahs', 'has_wool', 'beh_-_eats_grass', 'is_edible', 'an_animal', 'used_for_wool', 'is_fluffy', 'used_by_clipping/shearing', 'has_4_legs', 'has_fur', 'is_black', 'is_soft', 'eaten_as_meat', 'lives_on_farms', 'requires_shepherds']
****************
predictions for lamb
****************
../trained_models/model.ffnn.mc_rae_real.1k.50epochs.0.5dropout.lr1e-4.hsize300
      has_4_legs
      beh_-_lays_eggs
      beh_-_eats
      is_large
      has_feathers
      an_animal
      is_edible
      a_vegetable
      a_bird
      has_a_beak
../trained_models/model.plsr.mc_rae_real.1k.50components.500max_iters
      has_a_tail
      has_4_legs
      has_a_beak
      a_vegetable
      is_large
      is_white
      a_bird
      an_animal
      has_feathers
      is_edible
../trained_models/model.modabs.mc_rae_real.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4
predicting features for  lamb
      has_wings
      has_legs
      beh_-_

In [52]:
df = pd.DataFrame.from_records(records) 
df

,word,model,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17
0,lamb,gold,is_white,beh_-_bahs,has_wool,beh_-_eats_grass,is_edible,an_animal,used_for_wool,is_fluffy,used_by_clipping/shearing,has_4_legs,has_fur,is_black,is_soft,eaten_as_meat,lives_on_farms,requires_shepherds,NaN,NaN
1,lamb,ffnn,has_4_legs,beh_-_lays_eggs,beh_-_eats,is_large,has_feathers,an_animal,is_edible,a_vegetable,a_bird,has_a_beak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,lamb,plsr,has_a_tail,has_4_legs,has_a_beak,a_vegetable,is_large,is_white,a_bird,an_animal,has_feathers,is_edible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lamb,modabs,has_wings,has_legs,beh_-_lays_eggs,lives_on_farms,has_a_beak,is_edible,beh_-_eats,has_feathers,an_animal,a_bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bullet,gold,used_for_killing,used_in_guns,is_fast,made_of_metal,used_by_firing_from_gun,is_small,has_a_pointed_end,used_by_hunters,used_by_the_police,a_gun,made_of_lead,requires_gunpowder,used_as_a_projectile,used_for_injuring,NaN,NaN,NaN,NaN
5,bullet,ffnn,used_for_hunting,is_loud,a_gun,inbeh_-_fires_bullets,used_for_war,is_dangerous,inbeh_-_fires,used_for_killing,a_weapon,made_of_metal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bullet,plsr,used_by_the_police,a_gun,made_of_metal,used_for_war,a_weapon,used_for_killing,is_dangerous,is_loud,is_black,has_a_trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,bullet,modabs,is_round,used_for_hunting,is_loud,used_for_protection,a_weapon,inbeh_-_fires_bullets,inbeh_-_fires,made_of_metal,used_for_killing,is_dangerous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,crayon,gold,used_for_colouring,made_of_wax,used_by_children,different_colours,used_for_drawing,is_breakable,used_for_colouring_books,made_by_Crayola,comes_in_boxes,is_colourful,used_for_drawing_pictures,is_meltable,used_for_writing,NaN,NaN,NaN,NaN,NaN
9,crayon,ffnn,used_for_cutting,tastes_sweet,is_long,a_tool,is_sharp,made_of_metal,a_fruit,has_a_handle,is_small,made_of_wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Buchanana hs a different dataset, therefore different test word so we have to refetch those and then sample and run this again

In [18]:
# first sample 5 random words from the test set

feature_norms = BuchananFeatureNorms('../data/external/buchanan/cue_feature_words.csv')
layer = 8
clusters = 5
embedding_file = '../data/processed/multipro_embeddings/layer'+ str(layer) + 'clusters' + str(clusters) + '.txt'
embs = read_multiprototype_embeddings(embedding_file, layer=layer, num_clusters=clusters)

train, dev, test = classifier_main.prepare_data(feature_norms, embs)

test_words = []
for word in test:
    test_words.append(word)

test_words

Read in 12348 vectors of size 5 X 768
Starting with 3550 training words


['how',
 'clear',
 'witch',
 'husband',
 'closing',
 'saint',
 'dusk',
 'captain',
 'chorus',
 'virgin',
 'handicap',
 'knees',
 'similar',
 'ton',
 'unaware',
 'soda',
 'talent',
 'outgoing',
 'vacancy',
 'marines',
 'slip',
 'monarch',
 'beets',
 'bacteria',
 'dominate',
 'eat',
 'overpower',
 'fifty',
 'fast',
 'warning',
 'coat',
 'magician',
 'snooze',
 'glow',
 'choice',
 'crane',
 'ceiling',
 'difficult',
 'intoxicate',
 'donkey',
 'pliers',
 'flap',
 'billboard',
 'veil',
 'nuclear',
 'brook',
 'penguin',
 'biscuit',
 'rhythm',
 'fascism',
 'thesaurus',
 'failure',
 'annoy',
 'peg',
 'couple',
 'closet',
 'careful',
 'value',
 'recycle',
 'recliner',
 'entry',
 'chapel',
 'race',
 'personal',
 'handbag',
 'alive',
 'grocery',
 'roast',
 'shine',
 'camping',
 'valor',
 'parched',
 'presence',
 'grounds',
 'luck',
 'fireplace',
 'cutthroat',
 'album',
 'sticky',
 'float',
 'cobbler',
 'pineapple',
 'match',
 'blackbird',
 'cap_bottle',
 'trash',
 'fair',
 'stress',
 'zit',
 'perc

In [19]:
from random import sample
sample(test_words, 5)

['foundation', 'steel', 'ahead', 'lawyer', 'husk']

In [22]:
buchanan_test_words_sample = ['foundation', 'steel', 'ahead', 'lawyer', 'husk']

In [26]:

# buchanan

models = [
    '../trained_models/model.ffnn.buchanan.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.buchanan.1k.300components.500max_iters',
    '../trained_models/model.modabs.buchanan.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4',
]
records = []

# need to get buchanan test words

for word in buchanan_test_words_sample:
    print("****************")
    print("gold features for %s" % word)
    print("****************")
    golds = feature_norms.get_features(word)
    print(golds)
    record = {}
    record["word"] = word
    record["model"] = "gold"
    for i in range(0, len(golds)):
        gold = golds[i]
        record["feat_" + str(i)] = gold
    records.append(record)
    
    print("****************")
    print("predictions for %s" % word)
    print("****************")
    for save_path in models:
        model = torch.load(save_path)
        print(save_path)

        record = {}
        record["word"] = word
        record["model"] = save_path.split(".")[3]
        feats = model.predict_top_n_features(word, 10)
        for i in range(0, len(feats)):
            feat = feats[i]
            record["feat_" + str(i)] = feat
            
            print("     ", feat)
        records.append(record)
        


****************
gold features for foundation
****************
['base', 'begin', 'bottom', 'build', 'makeup', 'support']
****************
predictions for foundation
****************
../trained_models/model.ffnn.buchanan.1k.50epochs.0.5dropout.lr1e-4.hsize300
      together
      part
      school
      place
      large
      wear
      money
      group
      build
      person
../trained_models/model.plsr.buchanan.1k.300components.500max_iters
      water
      create
      need
      hard
      place
      large
      walk
      group
      person
      build
../trained_models/model.modabs.buchanan.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4
predicting features for  foundation
      person
      bottom
      part
      up
      place
      center
      point
      middle
      money
      spine
****************
gold features for steel
****************
['alloy', 'build', 'carbon', 'hard', 'iron', 'metal', 'strong']
****************
predictions for steel
****************
../trained_models

In [27]:
df = pd.DataFrame.from_records(records) 
df

,word,model,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10
0,foundation,gold,base,begin,bottom,build,makeup,support,NaN,NaN,NaN,NaN,NaN
1,foundation,ffnn,together,part,school,place,large,wear,money,group,build,person,NaN
2,foundation,plsr,water,create,need,hard,place,large,walk,group,person,build,NaN
3,foundation,modabs,person,bottom,part,up,place,center,point,middle,money,spine,NaN
4,steel,gold,alloy,build,carbon,hard,iron,metal,strong,NaN,NaN,NaN,NaN
5,steel,ffnn,heave,hold,store,tool,large,material,hard,metal,wood,build,NaN
6,steel,plsr,heave,large,brown,shine,strong,hard,color,metal,material,build,NaN
7,steel,modabs,belt,silver,color,brown,material,build,heave,hard,shine,metal,NaN
8,ahead,gold,advance,before,behind,front,future,NaN,NaN,NaN,NaN,NaN,NaN
9,ahead,ffnn,high,tree,car,time,leave,wood,direct,end,fast,move,NaN


# ANd we can do the same for binder..... bc why not

In [32]:
# first sample 5 random words from the test set

feature_norms = BinderFeatureNorms('data/external/binder_word_ratings/WordSet1_Ratings.csv')
layer = 8
clusters = 5
embedding_file = '../data/processed/multipro_embeddings/layer'+ str(layer) + 'clusters' + str(clusters) + '.txt'
embs = read_multiprototype_embeddings(embedding_file, layer=layer, num_clusters=clusters)

train, dev, test = classifier_main.prepare_data(feature_norms, embs)

test_words = []
for word in test:
    test_words.append(word)

test_words

Read in 12348 vectors of size 5 X 768
Starting with 428 training words


['delivered',
 'tax',
 'carrot',
 'army',
 'delirium',
 'trombone',
 'banana',
 'eggplant',
 'medicine',
 'knowledge',
 'mercy',
 'goldfish',
 'prison',
 'rumor',
 'muscle',
 'tree',
 'ant',
 'engineer',
 'sick',
 'musical',
 'scooter',
 'hierarchy',
 'watched',
 'feather',
 'feared',
 'worked',
 'farm',
 'treaty',
 'held',
 'carried',
 'honey',
 'testimony',
 'cathedral',
 'circus',
 'reporter',
 'terrorist',
 'wrote',
 'bee',
 'etiquette',
 'dark',
 'visited',
 'jury',
 'ire',
 'fee',
 'irony',
 'businessman',
 'keyboard',
 'gasp',
 'crossed',
 'hygiene',
 'loud',
 'mouth',
 'read']

In [33]:
from random import sample
sample(test_words, 5)

['crossed', 'circus', 'testimony', 'farm', 'tax']

In [34]:
binder_test_words_sample = ['crossed', 'circus', 'testimony', 'farm', 'tax']

In [41]:

# binder

models = [
    #'../trained_models/model.ffnn.mc_rae_real.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.ffnn.mc_rae_real.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.plsr.mc_rae_real.1k.50components.500max_iters',
    #'../trained_models/model.plsr.mc_rae_real.glove.100components.500max_iters',
    #'../trained_models/model.modabs.mc_rae_real.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4',
    #'../trained_models/model.modabs.mc_rae_real.glove.mu1_1.mu2_0.1.mu3_1e-07.mu4_5.nnk_5',
    '../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.1k.300components.500max_iters',
    '../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]


records = []

# need to get binder test words

for word in binder_test_words_sample:
    print("****************")
    print("gold features for %s" % word)
    print("****************")
    golds = feature_norms.top_n(word, 10)
    print(golds)
    record = {}
    record["word"] = word
    record["model"] = "gold"
    for i in range(0, len(golds)):
        gold = golds[i]
        record["feat_" + str(i)] = gold
    records.append(record)
    
    print("****************")
    print("predictions for %s" % word)
    print("****************")
    for save_path in models:
        model = torch.load(save_path)
        print(save_path)

        record = {}
        record["word"] = word
        record["model"] = save_path.split(".")[3]
        feats = model.predict_top_n_features(word, 10)
        for i in range(0, len(feats)):
            feat = feats[i]
            record["feat_" + str(i)] = feat
            
            print("     ", feat)
        records.append(record)
        


****************
gold features for crossed
****************
['Taste', 'Smell', 'Bright', 'Music', 'Disgusted', 'Color', 'Speech', 'Sad', 'Surprised', 'Dark']
****************
predictions for crossed
****************
../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300
      Consequential
      Social
      Human
      Vision
      Self
      Path
      Body
      Biomotion
      Motion
      Benefit
../trained_models/model.plsr.binder.1k.300components.500max_iters
      Self
      Pattern
      Cognition
      Head
      Away
      Texture
      Small
      Angry
      Near
      Body
../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4
predicting features for  crossed
      Human
      Self
      LowerLimb
      Benefit
      Consequential
      Path
      UpperLimb
      Body
      Biomotion
      Vision
****************
gold features for circus
****************
['Temperature', 'Pain', 'Self', 'Touch', 'Disgusted', 'Texture', 'Needs', 'Sa

In [42]:
df = pd.DataFrame.from_records(records) 
df

,word,model,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9
0,crossed,gold,Taste,Smell,Bright,Music,Disgusted,Color,Speech,Sad,Surprised,Dark
1,crossed,ffnn,Consequential,Social,Human,Vision,Self,Path,Body,Biomotion,Motion,Benefit
2,crossed,plsr,Self,Pattern,Cognition,Head,Away,Texture,Small,Angry,Near,Body
3,crossed,modabs,Human,Self,LowerLimb,Benefit,Consequential,Path,UpperLimb,Body,Biomotion,Vision
4,circus,gold,Temperature,Pain,Self,Touch,Disgusted,Texture,Needs,Sad,Small,Angry
5,circus,ffnn,Sound,Landmark,Arousal,Happy,Pleasant,Social,Scene,Large,Attention,Vision
6,circus,plsr,Near,Sad,Music,Landmark,Away,Shape,Cognition,Pattern,Dark,Pain
7,circus,modabs,Sound,Arousal,Happy,Pleasant,Social,Scene,Landmark,Large,Attention,Vision
8,testimony,gold,Small,Bright,Smell,Shape,Temperature,Weight,Dark,Color,Pattern,Touch
9,testimony,ffnn,Pleasant,Benefit,Sound,Consequential,Communication,Arousal,Social,Audition,Attention,Vision
